# Insalling Airbyte and Metabase on EC2 Instance

<P>It is recommended to use a t2.medium to run Airbyte properly.  <br>Anything smaller may cause the instance to crash or time-out</p>

## 1 - Install docker and docker compose

#### Step 1 - Install docker

In [ ]:
sudo apt update
sudo apt upgrade -y
sudo apt install docker.io
# Check that docker is installed
docker --version

#### After running upgrade-y you may see a pink screen that says 'Pending Kernel Upgrade'

<img src="Images/KernelUpgrade_800.png" alt="KernelUpgradeNeeded" width="650ptx">

#### If you do just make sure that the OK button is highlighted and hit enter.  We will fix this in a later step.

#### Step 2 - Install docker compose (manually)

In [ ]:
# Set the docker home folder and create a cli-plugins directory for docker compose
DOCKER_CONFIG=${DOCKER_CONFIG:-$HOME/.docker} 
mkdir -p $DOCKER_CONFIG/cli-plugins
# download docker compose into the cli-plugins directory 
# and change permissions to allow execution
curl -SL https://github.com/docker/compose/releases/download/v2.18.1/docker-compose-linux-x86_64 -o $DOCKER_CONFIG/cli-plugins/docker-compose
chmod +x $DOCKER_CONFIG/cli-plugins/docker-compose
# check that docker compose is installed
docker compose version

#### Step 3 - Confirm that you can use docker

In [ ]:
docker ps -a

`Got permission denied while trying to connect to the Docker daemon socket at 
unix:///var/run/docker.sock: Get "http://%2Fvar%2Frun%2Fdocker.sock/v1.24/containers/json?all=1": 
dial unix /var/run/docker.sock: connect: permission denied`

<p>At this point. You may notice that you need to run sudo before you can useany docker commands.  When you installed docker.io, a docker group was automatically created with only root access. To run the docker daemon without having to enter sudo every time add your current user to the docker group.</p>

In [ ]:
sudo usermod -aG docker $USER

<p> in order for this to take effect on a EC2 instance, you must reboot the instance.  Doing so will do two things. 1) it will update the kernel (the pink screen we encountered before), and evaluate the group change to take effect.</p>

In [ ]:
sudo reboot
# run this and wait a few moments for the system to reboot.

In [ ]:
# after logging back in try to confirm that you can use docker again.
docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES

# If you want, you can also run a test pull by running the following command.  
# If everything is installed properly it will pull the image and run 'hello world' in the terminal
docker run hello-world

## 2 - Install Airbyte

#### Step 1 - Install Airbyte

In [ ]:
# cone airbyte repository to your home directory.  This will create a folder named airbyte
git clone https://github.com/airbytehq/airbyte.git
cd airbyte
# running the following command for the first time will initiate docker to pull and run the airbyte image
./run-ab-platform.sh

<p>You will see the images being pulled and upon completion you'll see "Airbyte containers are running".  This may take a few minutes</p>

username `airbyte` and password `password`
url: ip-of-ec2:8000

#### Step 2 - Validate the Installation

1. First you should log out of the instance by running<br>
`exit`
2. Next ssh back into your instance, however we are going to make a couple of changes. Typically your instance connection would be:<br>
`ssh -i <SSH_KEY.pem> ubuntu@<INSTANCE_IP>`<br>
However we want to view airbyte in our local browser.  To do so ssh into your instance using the following command:<br>`ssh -i <SSH_KEY.pem> -L 8000:localhost:8000 -N -f ubuntu@<INSTANCE_IP>`<br>
See MAN SSH for details on the flags used here and all other available tags.<br> *Note that the -N flag causes the terminal to not run the shell so you will stay in your local terminal.
3. Now in your local browser, visit `http://localhost:8000`. If your installation worked correctly you will see a login popup for you to enter your username and password.
username `airbyte` and password `password`<br>
If all worked well you will see the following screen:<br>

<img src="Images/Airbyte.png" width="650ptx" />

## 3 - Install Metabase

#### Before you start this portion, make sure that your EC2 outbound security rules has CustomTCP Port 3000 open anywhere 0.0.0.0/0	

#### Step 1 - Pull the Metabase docker image


In [ ]:
docker pull metabase/metabase:latest

#### Step 2 - Run the docker container

In [ ]:
docker run -d -p 3000:3000 --name metabase metabase/metabase
# if you want to see the logs as metabase installs then run.  control + c to quit.
docker logs -f metabase

The above command will launch Metabase server on port 3000 by default
*Note You will have to wait a few moments for the server to fully start up.

Or, if you want to store the metadata of Metabase in your own database, you can use the following command to run the docker container:

In [ ]:
docker run -d -p 3000:3000 \
  -e "MB_DB_TYPE=postgres" \
  -e "MB_DB_DBNAME=<metabasea db>" \
  -e "MB_DB_PORT=5432" \
  -e "MB_DB_USER=name" \
  -e "MB_DB_PASS=password" \
  -e "MB_DB_HOST=my-database-host" \
   --name metabase metabase/metabase

#### Step 3 - Validate the Installation

Exactly like we validated for Airbyte, complete the following steps to check your metabase installation
1. First you should log out of the instance by running<br>
`exit`
2. ssh into your instance using the following command:
<br>`ssh -i <SSH_KEY.pem> -L 3000:localhost:3000 -N -f ubuntu@<INSTANCE_IP>`<br>
3. In your local browser, visit `http://localhost:3000`. If your installation worked correctly you will see the following page:

<img src="Images/Metabase.png" width="650ptx" />